# Entraînement du modèle de classification des prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour entraîner le modèle de classification des prunes africaines en utilisant le jeu de données Kaggle "African Plums Dataset".

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt
    !pip install kaggle

## 2. Configuration de l'API Kaggle

Pour télécharger le jeu de données Kaggle, nous devons configurer l'API Kaggle si ce n'est pas déjà fait dans le notebook de préparation des données.

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import json
import time
import shutil

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import load_and_prepare_two_stage_data
from models.model_architecture import get_model, TwoStageModel
from scripts.train_two_stage import train_model, evaluate_model, plot_training_history

# Définir les chemins des données
if IN_COLAB:
    # Dans Colab, créer les répertoires dans le dossier du projet cloné
    DATA_ROOT = "data/raw"
    KAGGLE_DIR = "data/kaggle"
    MODELS_DIR = "models/saved"
else:
    # En local
    DATA_ROOT = "../data/raw"
    KAGGLE_DIR = "../data/kaggle"
    MODELS_DIR = "../models/saved"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Créer les répertoires s'ils n'existent pas
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(KAGGLE_DIR, exist_ok=True)

# Définir les paramètres d'entraînement
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 2 if IN_COLAB else 4  # Réduire le nombre de workers dans Colab
LEARNING_RATE = 0.001
NUM_EPOCHS = 10 if IN_COLAB else 25  # Réduire le nombre d'époques dans Colab pour accélérer
EARLY_STOPPING_PATIENCE = 3 if IN_COLAB else 7  # Réduire la patience dans Colab
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

In [ ]:
# Configuration de l'API Kaggle si nécessaire
if IN_COLAB:
    from google.colab import files
    
    # Vérifier si le fichier kaggle.json existe déjà
    kaggle_config_exists = os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json'))
    
    if not kaggle_config_exists:
        print("Veuillez télécharger votre fichier kaggle.json pour l'authentification Kaggle.")
        print("Vous pouvez le générer sur https://www.kaggle.com/account dans la section 'API'.")
        
        # Télécharger le fichier kaggle.json
        uploaded = files.upload()
        
        # Créer le répertoire .kaggle s'il n'existe pas
        os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
        
        # Déplacer le fichier kaggle.json vers le répertoire .kaggle
        if 'kaggle.json' in uploaded:
            shutil.move('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))
            # Définir les permissions appropriées
            os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 600)
            print("Fichier kaggle.json configuré avec succès.")
        else:
            print("Erreur: Le fichier kaggle.json n'a pas été téléchargé.")
    else:
        print("Le fichier kaggle.json existe déjà.")

## 3. Vérification et préparation des données

Vérifions si les données du jeu de données Kaggle "African Plums Dataset" sont déjà préparées. Si ce n'est pas le cas, nous les téléchargerons et les préparerons.

In [ ]:
import zipfile

def download_kaggle_dataset(force_download=False):
    """Télécharge le jeu de données Kaggle 'African Plums Dataset'."""
    # Vérifier si le jeu de données a déjà été téléchargé
    dataset_zip = os.path.join(KAGGLE_DIR, 'african-plums-dataset.zip')
    if os.path.exists(dataset_zip) and not force_download:
        print(f"Le jeu de données a déjà été téléchargé dans {dataset_zip}.")
        return dataset_zip
    
    print("Téléchargement du jeu de données Kaggle 'African Plums Dataset'...")
    try:
        # Télécharger le jeu de données
        !kaggle datasets download -d arnaudfadja/african-plums-quality-and-defect-assessment-data -p {KAGGLE_DIR}
        print(f"Jeu de données téléchargé avec succès dans {dataset_zip}.")
        return dataset_zip
    except Exception as e:
        print(f"Erreur lors du téléchargement du jeu de données: {e}")
        return None

def extract_and_organize_dataset(dataset_zip, force_extract=False):
    """Extrait et organise le jeu de données Kaggle pour notre modèle."""
    if not os.path.exists(dataset_zip):
        print(f"Le fichier {dataset_zip} n'existe pas.")
        return False
    
    # Vérifier si les données ont déjà été extraites
    extracted_dir = os.path.join(KAGGLE_DIR, 'extracted')
    if os.path.exists(extracted_dir) and not force_extract:
        print(f"Le jeu de données a déjà été extrait dans {extracted_dir}.")
    else:
        print(f"Extraction du jeu de données...")
        os.makedirs(extracted_dir, exist_ok=True)
        
        # Extraire le fichier zip
        with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
            zip_ref.extractall(extracted_dir)
        
        print(f"Jeu de données extrait avec succès dans {extracted_dir}.")
    
    # Organiser les données pour notre modèle
    print("Organisation des données pour notre modèle...")
    
    # Vérifier la structure du jeu de données extrait
    print("Structure du jeu de données extrait:")
    !find {extracted_dir} -type d | sort
    
    # Mapper les classes du jeu de données Kaggle aux classes de notre modèle
    # Selon la description, les classes sont: bruised, cracked, rotten, spotted, unaffected, unripe
    class_mapping = {
        'bruised': 'bruised',
        'cracked': 'cracked',
        'rotten': 'rotten',
        'spotted': 'spotted',
        'unaffected': 'unaffected',
        'unripe': 'unripe'
    }
    
    # Créer les répertoires pour les classes de prunes
    for cls in class_mapping.values():
        os.makedirs(os.path.join(PLUM_DATA_DIR, cls), exist_ok=True)
    
    # Créer le répertoire pour les non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    os.makedirs(non_plum_dir, exist_ok=True)
    
    # Copier les images dans les répertoires appropriés
    dataset_dir = os.path.join(extracted_dir, 'african_plums_dataset')
    if os.path.exists(dataset_dir):
        # Parcourir les sous-répertoires du jeu de données
        for src_cls, dst_cls in class_mapping.items():
            src_dir = os.path.join(dataset_dir, src_cls)
            dst_dir = os.path.join(PLUM_DATA_DIR, dst_cls)
            
            if os.path.exists(src_dir):
                # Copier les images
                print(f"Copie des images de {src_dir} vers {dst_dir}...")
                !cp -r {src_dir}/* {dst_dir}/
            else:
                print(f"Le répertoire {src_dir} n'existe pas.")
        
        print("Images copiées avec succès.")
        return True
    else:
        print(f"Le répertoire {dataset_dir} n'existe pas.")
        return False

def download_non_plum_images(num_images=100):
    """Télécharge des images non-prunes à partir d'un autre jeu de données Kaggle."""
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    
    # Vérifier si des images non-prunes existent déjà
    existing_images = [f for f in os.listdir(non_plum_dir) if os.path.isfile(os.path.join(non_plum_dir, f))]
    if existing_images:
        print(f"Des images non-prunes existent déjà ({len(existing_images)} images).")
        return
    
    print("Téléchargement d'images non-prunes...")
    
    # Option 1: Télécharger des images de fruits (autres que des prunes) à partir d'un jeu de données Kaggle
    try:
        # Télécharger un jeu de données de fruits
        !kaggle datasets download -d moltean/fruits -p {KAGGLE_DIR}
        
        # Extraire le jeu de données
        fruits_zip = os.path.join(KAGGLE_DIR, 'fruits.zip')
        fruits_dir = os.path.join(KAGGLE_DIR, 'fruits')
        os.makedirs(fruits_dir, exist_ok=True)
        
        with zipfile.ZipFile(fruits_zip, 'r') as zip_ref:
            zip_ref.extractall(fruits_dir)
        
        # Sélectionner des images aléatoires (excluant les prunes)
        import glob
        all_fruit_images = []
        for fruit_dir in glob.glob(os.path.join(fruits_dir, 'fruits-360/Training/*')):
            fruit_name = os.path.basename(fruit_dir).lower()
            if 'plum' not in fruit_name and 'prune' not in fruit_name:
                all_fruit_images.extend(glob.glob(os.path.join(fruit_dir, '*.jpg')))
        
        # Sélectionner un sous-ensemble aléatoire
        if all_fruit_images:
            selected_images = random.sample(all_fruit_images, min(num_images, len(all_fruit_images)))
            
            # Copier les images sélectionnées
            for i, img_path in enumerate(selected_images):
                dst_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
                shutil.copy(img_path, dst_path)
            
            print(f"{len(selected_images)} images non-prunes copiées avec succès.")
        else:
            print("Aucune image de fruit trouvée.")
            return False
        
        return True
    except Exception as e:
        print(f"Erreur lors du téléchargement d'images non-prunes: {e}")
        
        # Option 2: Créer des images synthétiques si le téléchargement échoue
        print("Création d'images non-prunes synthétiques...")
        
        for i in range(num_images):
            # Couleur aléatoire qui n'est pas proche des couleurs de prune
            color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
            
            # Créer une image
            from PIL import Image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner une forme aléatoire (carré ou triangle)
            shape = random.choice(['square', 'triangle'])
            
            if shape == 'square':
                # Dessiner un carré
                size = random.randint(100, 150)
                top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
                
                for x in range(top_left[0], top_left[0] + size):
                    for y in range(top_left[1], top_left[1] + size):
                        if 0 <= x < 224 and 0 <= y < 224:
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
            else:
                # Dessiner un triangle
                p1 = (random.randint(50, 174), random.randint(50, 174))
                p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
                p3 = (p1[0] - random.randint(0, 30), p2[1])
                
                # Remplir le triangle (algorithme simple)
                min_x = min(p1[0], p2[0], p3[0])
                max_x = max(p1[0], p2[0], p3[0])
                min_y = min(p1[1], p2[1], p3[1])
                max_y = max(p1[1], p2[1], p3[1])
                
                for x in range(min_x, max_x + 1):
                    for y in range(min_y, max_y + 1):
                        if 0 <= x < 224 and 0 <= y < 224:
                            # Vérification simple si le point est dans le triangle
                            if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                                # Ajouter du bruit à chaque pixel
                                pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                                pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
            img.save(img_path)
        
        print(f"{num_images} images non-prunes synthétiques créées avec succès.")
        return True

# Vérifier si les données sont déjà préparées
def check_data_availability():
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Téléchargement des données nécessaire.")
        return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Téléchargement des données nécessaire.")
        return False
    
    # Vérifier s'il y a des images dans les répertoires
    plum_images_count = sum(len([f for f in os.listdir(os.path.join(PLUM_DATA_DIR, cls)) 
                                if os.path.isfile(os.path.join(PLUM_DATA_DIR, cls, f))]) for cls in plum_classes)
    non_plum_images_count = len([f for f in os.listdir(non_plum_dir) if os.path.isfile(os.path.join(non_plum_dir, f))])
    
    if plum_images_count == 0 or non_plum_images_count == 0:
        print(f"Pas assez d'images dans les répertoires. Téléchargement des données nécessaire.")
        return False
    
    print(f"Données disponibles: {plum_images_count} images de prunes dans {len(plum_classes)} classes, {non_plum_images_count} images non-prunes.")
    return True

# Vérifier et préparer les données si nécessaire
data_available = check_data_availability()

if not data_available:
    print("Préparation des données...")
    # Télécharger et préparer le jeu de données
    dataset_zip = download_kaggle_dataset(force_download=False)
    if dataset_zip:
        success = extract_and_organize_dataset(dataset_zip, force_extract=False)
        if success:
            print("Jeu de données Kaggle préparé avec succès pour notre modèle.")
            # Télécharger des images non-prunes
            download_non_plum_images(num_images=100)
            data_available = check_data_availability()
        else:
            print("Erreur lors de la préparation du jeu de données Kaggle.")
    else:
        print("Erreur lors du téléchargement du jeu de données Kaggle.")

## 4. Chargement des données

Utilisons la fonction `load_and_prepare_two_stage_data` du module `data_preprocessing` pour charger les données.

In [ ]:
if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord préparer les données.")

## 5. Création des modèles

Utilisons la fonction `get_model` du module `model_architecture` pour créer les modèles.

In [ ]:
if data_available and 'detection_class_names' in locals() and 'classification_class_names' in locals():
    # Créer le modèle de détection
    detection_model = get_model(
        model_name='lightweight', 
        num_classes=len(detection_class_names), 
        base_model='mobilenet_v2', 
        pretrained=True
    )
    
    # Créer le modèle de classification
    classification_model = get_model(
        model_name='standard', 
        num_classes=len(classification_class_names),
        base_model='resnet18', 
        pretrained=True
    )
    
    # Afficher les informations sur les modèles
    print("=== Modèle de détection ===")
    print(f"Type: {detection_model.__class__.__name__}")
    print(f"Informations: {detection_model.get_model_info()}")
    
    print("\n=== Modèle de classification ===")
    print(f"Type: {classification_model.__class__.__name__}")
    print(f"Informations: {classification_model.get_model_info()}")
    
    # Déplacer les modèles sur le device
    detection_model = detection_model.to(device)
    classification_model = classification_model.to(device)

## 6. Entraînement du modèle de détection

Utilisons la fonction `train_model` du module `train_two_stage` pour entraîner le modèle de détection.

In [ ]:
if data_available and 'detection_train_loader' in locals() and 'detection_val_loader' in locals() and 'detection_model' in locals():
    try:
        print("=== Entraînement du modèle de détection ===\n")
        
        # Définir la fonction de perte et l'optimiseur
        detection_criterion = nn.CrossEntropyLoss()
        detection_optimizer = optim.Adam(detection_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        detection_scheduler = ReduceLROnPlateau(detection_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de détection
        detection_history = train_model(
            detection_model, 
            detection_train_loader, 
            detection_val_loader, 
            detection_criterion, 
            detection_optimizer, 
            detection_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(detection_history, save_dir=MODELS_DIR, model_name="detection")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de détection: {e}")

## 7. Entraînement du modèle de classification

Utilisons la fonction `train_model` du module `train_two_stage` pour entraîner le modèle de classification.

In [ ]:
if data_available and 'classification_train_loader' in locals() and 'classification_val_loader' in locals() and 'classification_model' in locals():
    try:
        print("=== Entraînement du modèle de classification ===\n")
        
        # Définir la fonction de perte et l'optimiseur
        classification_criterion = nn.CrossEntropyLoss()
        classification_optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        classification_scheduler = ReduceLROnPlateau(classification_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de classification
        classification_history = train_model(
            classification_model, 
            classification_train_loader, 
            classification_val_loader, 
            classification_criterion, 
            classification_optimizer, 
            classification_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(classification_history, save_dir=MODELS_DIR, model_name="classification")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de classification: {e}")

## 8. Évaluation des modèles

Utilisons la fonction `evaluate_model` du module `train_two_stage` pour évaluer les modèles.

In [ ]:
# Évaluer le modèle de détection
if data_available and 'detection_test_loader' in locals() and 'detection_class_names' in locals():
    try:
        print("=== Évaluation du modèle de détection ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth'), map_location=device))
        detection_model = detection_model.to(device)
        
        # Évaluer le modèle
        detection_criterion = nn.CrossEntropyLoss()
        detection_metrics = evaluate_model(
            detection_model, 
            detection_test_loader, 
            detection_criterion, 
            device, 
            detection_class_names,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de détection: {e}")

In [ ]:
# Évaluer le modèle de classification
if data_available and 'classification_test_loader' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Évaluation du modèle de classification ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth'), map_location=device))
        classification_model = classification_model.to(device)
        
        # Évaluer le modèle
        classification_criterion = nn.CrossEntropyLoss()
        classification_metrics = evaluate_model(
            classification_model, 
            classification_test_loader, 
            classification_criterion, 
            device, 
            classification_class_names,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de classification: {e}")

## 9. Sauvegarde du modèle à deux étapes

Créons et sauvegardons le modèle à deux étapes complet.

In [ ]:
# Créer et sauvegarder le modèle à deux étapes
if data_available and 'detection_class_names' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Création du modèle à deux étapes ===\n")
        
        # Charger les meilleurs modèles
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth'), map_location=device))
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth'), map_location=device))
        
        # Créer le modèle à deux étapes
        two_stage_model = TwoStageModel(detection_model, classification_model, detection_threshold=0.7)
        
        # Sauvegarder les informations du modèle
        model_info = {
            'detection_classes': detection_class_names,
            'classification_classes': classification_class_names,
            'model_info': two_stage_model.get_model_info(),
            'img_size': IMG_SIZE,
            'date_created': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(os.path.join(MODELS_DIR, 'two_stage_model_info.json'), 'w') as f:
            json.dump(model_info, f, indent=4)
        
        print("Modèle à deux étapes créé et informations sauvegardées.")
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        print(f"Informations du modèle: {two_stage_model.get_model_info()}")
        
        # Dans Colab, permettre de télécharger les modèles entraînés
        if IN_COLAB:
            from google.colab import files
            print("\nVous pouvez télécharger les modèles entraînés ci-dessous:")
            files.download(os.path.join(MODELS_DIR, 'detection_best_acc.pth'))
            files.download(os.path.join(MODELS_DIR, 'classification_best_acc.pth'))
            files.download(os.path.join(MODELS_DIR, 'two_stage_model_info.json'))
    except Exception as e:
        print(f"Erreur lors de la création du modèle à deux étapes: {e}")

## 10. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes des modules `data_preprocessing`, `model_architecture` et `train_two_stage` pour :
1. Télécharger et préparer le jeu de données Kaggle "African Plums Dataset"
2. Charger les données
3. Créer les modèles de détection et de classification
4. Entraîner les modèles
5. Évaluer les performances
6. Créer et sauvegarder le modèle à deux étapes complet

Le modèle est maintenant prêt à être testé sur de nouvelles images dans le notebook suivant.